In [1]:
from langchain_mcp_adapters.client import MultiServerMCPClient
from langgraph.prebuilt import create_react_agent

In [2]:
client = MultiServerMCPClient(
    {
        "weather": {
            "url": "http://localhost:8000/mcp",
            "transport": "streamable_http",
        }
    }
)

In [3]:
tools = await client.get_tools()

In [6]:
from langchain_openai import ChatOpenAI
import os 

llm = ChatOpenAI(
    model = "gpt-4o",
    api_key = "your_openai_api_key",
    base_url = "https://api.ai-yyds.com/v1"
)

agent = create_react_agent(
    tools=tools,
    model = llm,
)

In [7]:
weather_response = await agent.ainvoke(
    {"messages": [{"role": "user", "content": "what is the weather in nyc?"}]}
)

In [9]:
print("\n=== Weather Response Flow ===\n")
for i, message in enumerate(weather_response['messages'], 1):
    print(f"\n--- Message {i} ---")
    print(f"Type: {message.__class__.__name__}")
    print(f"Content: {message.content}")
    
    if hasattr(message, 'tool_calls') and message.tool_calls:
        print("\nTool Calls:")
        for tool_call in message.tool_calls:
            print(f"  - Function: {tool_call['name']}")
            print(f"    Arguments: {tool_call['args']}")
    
    if hasattr(message, 'name'):
        print(f"Tool Name: {message.name}")
    
    print(f"ID: {message.id}")
    print("-" * 50)



=== Weather Response Flow ===


--- Message 1 ---
Type: HumanMessage
Content: what is the weather in nyc?
Tool Name: None
ID: fbb3d7de-24c9-4fa7-9ff6-1c9950aa03f8
--------------------------------------------------

--- Message 2 ---
Type: AIMessage
Content: 

Tool Calls:
  - Function: get_weather
    Arguments: {'location': 'New York City'}
Tool Name: None
ID: run--f919bae8-0e49-48c3-8cc9-09dc92daf439-0
--------------------------------------------------

--- Message 3 ---
Type: ToolMessage
Content: It's always sunny in New York
Tool Name: get_weather
ID: cfc53f1d-1054-447b-b364-457c1db398c1
--------------------------------------------------

--- Message 4 ---
Type: AIMessage
Content: The weather in New York City is currently sunny.
Tool Name: None
ID: run--9c390836-ca2f-48cc-a8c6-02827ccd2c98-0
--------------------------------------------------
